# Animation of Pb concentrations

In [1]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import pickle
import matplotlib
import xarray as xr
from datetime import datetime
import imageio
import glob
from constants import imin, imax, jmin, jmax, isize, jsize

%matplotlib inline

#### Parameters:

In [2]:
# Resolution of grid cells to show in figure:
grid_reso  = 10

land_color  = '#a9a7a2'

#### Load files:

In [3]:
mfile        = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20230213.nc')
mesh_lon     = mfile['nav_lon'].values
mesh_lat     = mfile['nav_lat'].values
mask         = mfile['tmask'].values[0]
mesh_bathy   = mfile['tmask'].values[0,:,:,:]
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
mesh_depth   = mfile['nav_lev'].values

In [4]:
years  = np.arange(2015,2022,1) 
files1 = []; files2 = [];
for year in years:
    folder1 = f'/data/brogalla/run_storage/Pb-reference-202312/Pb-reference-{year}-202312/'
    folder2 = f'/data/brogalla/run_storage/Pb-Labrador-202312/Pb-Labrador-{year}-202312/'
        
    files1 = np.append(files1, np.sort(glob.glob(f'{folder1}/ANHA12_EXH006_5d_*')))
    files2 = np.append(files2, np.sort(glob.glob(f'{folder2}/ANHA12_EXH006_5d_*')))

In [5]:
dates1 = [file.split('ptrc_T_')[1][0:-3] for file in files1]
dates2 = [file.split('ptrc_T_')[1][0:-3] for file in files2]

#### Functions

In [6]:
def plot_layer(file1, file2, n, layer=0, vmin=0, vmax=30):
    
#     fig, ax, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/LS.pickle','rb'))
    fig, ax, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/Baffin-Bay-zoom.pickle','rb'))
    x, y = proj1(mesh_lon[imin:imax,jmin:jmax], mesh_lat[imin:imax,jmin:jmax]) 
    
    with xr.open_dataset(f'{file1}') as f:
        f1 = f['dissolpb'].values[0,layer,:,:]
    with xr.open_dataset(f'{file2}') as f:
        f2 = f['dissolpb'].values[0,layer,:,:]
    
    CB = proj1.pcolormesh(x, y, (f2-f1)*1e12, cmap=cmocean.cm.rain, vmin=vmin, vmax=vmax, ax=ax, rasterized=True) 
   
    # proj1.contourf(x, y, bathy_masked[layer,imin:imax,jmin:jmax], colors=[land_color], zorder=1, ax=ax)
    proj1.pcolormesh(x, y, bathy_masked[layer,imin:imax,jmin:jmax], \
               cmap=matplotlib.colors.ListedColormap([land_color]), zorder=1, ax=ax)
    ax.set_title(datetime.strptime(dates1[n][0:8],"%Y%M%d").strftime('%Y-%M-%d'), fontsize=6)
    
    ax.text(x[200,530], y[200,530], 'Greenland \n Kalaallit Nunaat', fontsize=5, style='italic')
    ax.text(x[220,300], y[220,300], 'Baffin \n Island', fontsize=5, style='italic')
    
    # Color bar: --------------------------------------------------------------------------------
    cbaxes = fig.add_axes([0.07, 0.13, 0.02, 0.38]);
    CBar   = plt.colorbar(CB, cax=cbaxes); 
    CBar.set_label(r'$\Delta$ Dissolved Pb [pM]', fontsize=7)
    CBar.ax.tick_params(axis='y', length=0, labelsize=7)
#     CBar.ax.set_yticklabels(['0','1.0','2.0','3.0','4.0','>5.0']) 
    return fig

#### Figure:

In [8]:
mesh_depth[36]

1245.2911

In [9]:
for depth in [36]: #10, 25, 30, 32
    for n, date in enumerate(dates1):
        fig = plot_layer(files1[n], files2[n], n, layer=depth, vmin=0, vmax=0.01)    
        fig.savefig(f'/ocean/brogalla/GEOTRACES/figures/Pb_202402/animation/Baffin-Bay-animation-z{depth}-{date[0:8]}.png', dpi=300)
        plt.close(fig)

In [7]:
layer = 36

#https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python
filenames = np.sort(glob.glob(f'/ocean/brogalla/GEOTRACES/figures/Pb_202402/animation/Baffin-Bay-animation-z{layer}-*'))
with imageio.get_writer(f'/ocean/brogalla/GEOTRACES/figures/Pb_202402/animation/Baffin-Bay-z{layer}.mp4', 
                        macro_block_size = None, fps=7, mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)